Code snippet as appear in the slides. Not all of them are valid Python code.

In [1]:
import numpy as np

In [5]:
from typing import Callable

def f(*args, **kwargs):
    pass

def g(input_fn: Callable, *args, **kwargs) -> Callable:
    pass

new_fn = g(f)
# output = f(...)
# new_output = new_fn(...)

In [ ]:
from functools import partial

def power(x, k):
    return x ** k

square = partial(power, k=2)
square = lambda x: power(x, k=2)

In [ ]:
def g(input_fn: Callable, *args, **kwargs) -> Callable:
    # unpacking of args or kwargs
    b, c = args
    b_new = some_function(b)
    return lambda a: input_fn(a, b_new, c, **kwargs)

In [4]:
# Example motivated by https://jax.readthedocs.io/en/latest/jax-101/07-state.html
from collections import namedtuple
State = namedtuple('State', ['number', 'other'])
Output = namedtuple('Output', ['number', 'other', 'more_stuff'])

state = State(np.zeros(5), 1)

class StatefulClass:
    state: State
    def __init__(self, state):
        self.state = state

    def stateful_method(self, *args, **kwargs) -> Output:
        self.update_state(*args, **kwargs)
        return Output(np.zeros(10), 10, 'test')

    def update_state(self, *args, **kwargs) -> None:
        print('update state')
        self.state = State(np.ones(10), 5)

class StatelessClass:

    def stateless_method(self, state: State, *args, **kwargs) -> (Output, State):
        next_state = self.update_state(state, *args, **kwargs)
        return Output(np.zeros(10), 10, 'test'), next_state
    
    def update_state(state: State, *args, **kwargs) -> State:
        print('update state')
        return State(np.ones(10), 5)

init_state = State(np.zeros(5), 1)
fake_input = 'input_place_holder'
state_full_compute = StatefulClass(init_state)
print(state_full_compute.stateful_method(fake_input), state_full_compute.state)

state_less_compute = StatelessClass()
print(state_less_compute.stateless_method(init_state, fake_input))

update state
Output(number=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), other=10, more_stuff='test') State(number=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), other=5)
update state
(Output(number=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), other=10, more_stuff='test'), State(number=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), other=5))


In [ ]:
import jax
import jax.numpy as jnp
import jax.scipy as jsp

In [ ]:
import tensorflow_probability.substrates.jax as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

tf = tfp.tf2jax
tfl = tfp.tf2jax.linalg

In [ ]:
import jax
import jax.numpy as jnp
import jax.scipy.stats as stats
import numpy as np

import blackjax

observed = np.random.normal(10, 20, size=1_000)
def logdensity_fn(x):
    logpdf = stats.norm.logpdf(observed, x["loc"], x["scale"])
    return jnp.sum(logpdf)

# Build the kernel
step_size = 1e-3
inverse_mass_matrix = jnp.array([1., 1.])
nuts = blackjax.nuts(logdensity_fn, step_size, inverse_mass_matrix)

# Initialize the state
initial_position = {"loc": 1., "scale": 2.}
state = nuts.init(initial_position)

# Iterate
rng_key = jax.random.key(0)
step = jax.jit(nuts.step)
for _ in range(1_000):
    _, rng_key = jax.random.split(rng_key)
    state, _ = step(rng_key, state)

In [ ]:
ArrayTree = Union[Array, Iterable['ArrayTree'], Mapping[Any, 'ArrayTree']]


def prior_sampling(rng, num_samples) -> ArrayTree:
    ...

def prior_predictive_sampling(rng, num_samples) -> ArrayTree:
    ...

def posterior_sampling(rng, data, num_samples) -> ArrayTree:
    ...
    return posterior_samples

def posterior_predictive_sampling(
        rng, data, posterior_samples, num_samples) -> ArrayTree:
    ...